In [ ]:
"""
!pip install opencv-python
!pip install tensorflow
!pip install pillow
"""

import cv2
import PIL
import numpy as np
import tensorflow as tf

emoji_array=np.load('emoji.npy')
model=tf.keras.models.load_model('model.h5')

# создание окна трансляции с камеры
cv2.namedWindow('Emotion Detector')

# получение первой в списке камеры
cap=cv2.VideoCapture(0)

while True:
    # считывание кадров (frame) с камеры
    ret,frame=cap.read()

    # преобразование кадров в однокональные изображения
    frame_2=frame[0:len(frame[1]),0:len(frame[0]),1]/255.0

    # преобразование кадров в формат для PIL
    img=PIL.Image.fromarray(frame_2)

    # преобразование кадров в равносторонние изображения размером 80 на 80
    if img.size[0]>80 or img.size[1]>80:
        if img.size[0]>img.size[1]:
            left=(img.size[0]-img.size[1])/2
            right=img.size[0]-left
        else:
            left=(img.size[1]-img.size[0])/2
            right=img.size[1]-left

    img=img.crop((left,0,right,img.size[1])).resize((80,80))

    # добавление размерности для model.predict()
    img=np.asarray(img)[None,...]

    # предстказание
    img=emoji_array[np.argmax(model.predict(img))]

    # преобразование предстказания в трехканальное изображение
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

    # наложение изоображения предстказания на трансляцию камеры
    result=cv2.addWeighted(frame[320:480,480:640],1,img,0.5,0)
    frame[320:480, 480:640]=result

    # отрисовка изображения с трансляции
    cv2.imshow('Emotion Detector',frame)

    # остановка программы по нажатию escape
    if cv2.waitKey(1)==27:
        break

# освобождение камеры
cap.release()

# закрытие окна
cv2.destroyAllWindows()